<img src="../../images/routing-or-handoff.png" width="200">

Use Cases:

* Customer support systems: Routing queries to agents specialized in billing, technical support, or product information.
* Tiered LLM usage: Routing simple queries to faster, cheaper models (like Llama 3.1 8B) and complex or unusual questions to more capable models (like Gemini 1.5 Pro).
* Content generation: Routing requests for blog posts, social media updates, or ad copy to different specialized prompts/models.

source: https://www.philschmid.de/agentic-pattern

In [2]:
# ============================================================================
# SETUP: Import Dependencies
# ============================================================================
# This notebook demonstrates a routing agent pattern using OpenAI's structured
# outputs to intelligently classify user requests and route them to specialized handlers.

from openai import OpenAI
from src.fnHelpers import pydantic_to_openai_schema
from pydantic import BaseModel, Field
import json

# Initialize OpenAI client (requires OPENAI_API_KEY environment variable)
client = OpenAI()

In [3]:
# ============================================================================
# STEP 1: Define Pydantic Models for Structured Output
# ============================================================================
# These models ensure type-safe, validated responses from OpenAI.

class EmailDraft(BaseModel):
    """Structured email with recipient, subject, and body."""
    to: str = Field(description="Email recipient or placeholder name")
    subject: str = Field(description="Email subject line (under 60 characters)")
    body: str = Field(description="Email body text with professional tone")

class RoutingDecision(BaseModel):
    """Classification result for routing user requests."""
    category: str = Field(description="One of: 'email', 'calendar', or 'unclassified'")
    confidence: float = Field(description="Confidence score 0.0-1.0")
    reasoning: str = Field(description="Brief explanation of the classification")

# ============================================================================
# STEP 2: Define Worker Agents (Specialized Functions)
# ============================================================================
# Each agent handles a specific type of task.

def handle_email_task(task_details: str):
    """
    Email agent: Drafts professional emails using OpenAI with structured output.
    
    Uses Pydantic models to ensure consistent, validated email data.
    """
    print("\n--- 📧 Email Agent Activated ---")
    print(f"Received request: '{task_details}'")

    # Create the prompt for OpenAI
    prompt = f"""
    You are an expert email drafting assistant. Analyze the user's request and 
    generate a professional email draft.
    
    Requirements:
    - Use descriptive placeholders like "project_manager" or "hr_team" if recipients aren't specified
    - Keep the subject line concise (under 60 characters)
    - Write the body in professional tone with proper greeting and closing
    - Use line breaks (\\n) in the body for readability
    
    User Request: "{task_details}"
    
    Generate the email:"""

    try:
        # Convert Pydantic model to OpenAI schema
        email_schema = pydantic_to_openai_schema(EmailDraft, "email_draft")
        
        # Call OpenAI with structured output
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_schema", "json_schema": email_schema},
        )
        
        # Parse the response
        email_data = json.loads(response.choices[0].message.content)
        
        # Log the formatted version for debugging
        print("\n--- Generated Email Draft ---")
        print(f"To: {email_data['to']}")
        print(f"Subject: {email_data['subject']}")
        print(f"Body: {email_data['body'][:100]}...")
        
        # Return structured data
        return {
            "success": True,
            "email": email_data,
            "agent": "email_agent"
        }
        
    except Exception as e:
        print(f"🔴 Error in email agent: {e}")
        return {
            "success": False,
            "error": f"Failed to draft email: {str(e)}"
        }

In [4]:
handle_email_task("say hi to my friend.")


--- 📧 Email Agent Activated ---
Received request: 'say hi to my friend.'

--- Generated Email Draft ---
To: friend
Subject: A Friendly Hello!
Body: Dear Friend,

I hope this message finds you well. 

Just wanted to take a moment to say hi and see h...


{'success': True,
 'email': {'to': 'friend',
  'subject': 'A Friendly Hello!',
  'body': "Dear Friend,\n\nI hope this message finds you well. \n\nJust wanted to take a moment to say hi and see how you’re doing. It's always great to catch up with friends! \n\nLooking forward to hearing from you soon.\n\nBest regards,\n\n[Your Name]"},
 'agent': 'email_agent'}

In [5]:
# ============================================================================
# STEP 3: Router Agent (Powered by OpenAI)
# ============================================================================
# This is the core of the routing pattern. It uses OpenAI to intelligently
# classify requests and route them to the appropriate handler.

def openai_route_task(user_input: str):
    """
    Intelligently classifies user input using OpenAI and routes to appropriate handler.
    
    Returns a RoutingDecision with category, confidence, and reasoning.
    """
    print(f"🧠 OpenAI Router received: '{user_input}'")

    # Create the classification prompt
    prompt = f"""
    You are an intelligent routing agent. Classify the following user request into one of these categories:
    
    'email' - Requests related to drafting, sending, or replying to emails
    'calendar' - Requests about scheduling events, meetings, or reminders
    'unclassified' - Any other requests that don't fit above categories
    
    Provide your classification with confidence level and reasoning.
    
    User Request: "{user_input}"
    
    Classify this request:"""

    try:
        # Convert Pydantic model to OpenAI schema
        routing_schema = pydantic_to_openai_schema(RoutingDecision, "routing_decision")
        
        # Call OpenAI with structured output (temperature=0 for consistency)
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,  # Deterministic routing
            response_format={"type": "json_schema", "json_schema": routing_schema},
        )
        
        # Parse the structured response
        routing_data = json.loads(response.choices[0].message.content)
        category = routing_data["category"].lower()
        
        print(f"   Classification: '{category}' (confidence: {routing_data['confidence']})")
        print(f"   Reasoning: {routing_data['reasoning']}")
        
    except Exception as e:
        print(f"🔴 Error in router: {e}")
        category = "unclassified"  # Default to unclassified on error

    # Route based on classification
    if category == "email":
        return handle_email_task(user_input)
    elif category == "calendar":
        return handle_calendar_task(user_input)
    else:
        return handle_unclassified_task(user_input)

def handle_calendar_task(task_details: str):
    """
    Calendar agent: Handles scheduling and calendar-related requests.
    
    Note: This is a placeholder. In a production system, this would integrate
    with calendar APIs (Google Calendar, Outlook, etc.)
    """
    print("\n--- 📅 Calendar Agent Activated ---")
    print(f"Task: Create a calendar event from the request: '{task_details}'")
    print("Action: A calendar event would be created here using a calendar API.")
    print("----------------------------------\n")
    return "The event has been added to your calendar."

def handle_unclassified_task(task_details: str):
    """
    Fallback handler: Processes requests that don't fit into known categories.
    
    In a real system, this could log the request, escalate to a human,
    or trigger additional routing logic.
    """
    print("\n--- ❓ Unclassified Task Agent Activated ---")
    print(f"I'm not equipped to handle this request: '{task_details}'")
    print("Action: This could be logged for review or escalated to support.")
    print("-----------------------------------------\n")
    return "I'm not sure how to handle that request. Please try rephrasing or contact support."

User Input 1: schedule a meeting with Alex for Friday at 3pm -> This will be routed to handle_calendar_task.

User Input 2: draft an email to the team about the project update -> This will be routed to handle_email_task.

User Input 3: what is the weather today? -> This will be routed to handle_unclassified_task, demonstrating the need for a fallback.

In [6]:
# ============================================================================
# STEP 4: Main Application Loop (Safe Demo)
# ============================================================================
# The original infinite while-loop will hang in notebook environments because it
# waits for stdin. To keep the notebook runnable, we provide a demo runner with
# sample inputs. Set ENABLE_INTERACTIVE = True to use the interactive loop.

ENABLE_INTERACTIVE = False

sample_inputs = [
    "schedule a meeting with Alex for Friday at 3pm",
    "draft an email to the team about the project update",
    "what is the weather today?",
]

def run_demo_requests(requests):
    print("\n▶️ Running demo requests (non-interactive)...")
    for req in requests:
        print(f"\nUser: {req}")
        result = openai_route_task(req)
        print(f"Assistant: {result}")
    print("\n✅ Demo complete")

if ENABLE_INTERACTIVE:
    while True:
        user_command = None
        try:
            user_command = input("\n🤖 Personal Assistant (powered by OpenAI) is ready. Type 'quit' to exit: ")
        except KeyboardInterrupt:
            user_command = "quit"
        except EOFError:
            user_command = "quit"
        
        if user_command.lower() == 'quit':
            print("\n👋 Goodbye!")
            break

        result = openai_route_task(user_command)
        print(f"\n📝 Assistant's Final Response: {result}")
else:
    run_demo_requests(sample_inputs)



▶️ Running demo requests (non-interactive)...

User: schedule a meeting with Alex for Friday at 3pm
🧠 OpenAI Router received: 'schedule a meeting with Alex for Friday at 3pm'
   Classification: 'calendar' (confidence: 0.95)
   Reasoning: The request explicitly asks to schedule a meeting, which falls under the category of calendar-related activities.

--- 📅 Calendar Agent Activated ---
Task: Create a calendar event from the request: 'schedule a meeting with Alex for Friday at 3pm'
Action: A calendar event would be created here using a calendar API.
----------------------------------

Assistant: The event has been added to your calendar.

User: draft an email to the team about the project update
🧠 OpenAI Router received: 'draft an email to the team about the project update'
   Classification: 'email' (confidence: 0.95)
   Reasoning: The request explicitly asks to draft an email, which falls directly under the category of email-related tasks.

--- 📧 Email Agent Activated ---
Received req